## Постановка задачи
Загрузим данные и разделим выборку на обучающую/проверочную в соотношении 80/20.

Для обучающей выборки вычислим средние значения для веса, роста, индекса массы тела и возраста - для каждого из принятых решений. Предскажем оценку скоринга по близости данных средним значениям.

Проверим качество предсказания через F1-метрику и матрицу неточностей.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import cohen_kappa_score

data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB


### Разделение данных на обучающие и проверочные, 80/20

In [2]:
data_train, data_test = train_test_split(data, test_size=.2)
data_train.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
39329,52214,1,D4,26,0.487179,2,3,1,0.268657,0.727273,...,0,0,0,0,0,0,0,0,0,7
24166,32229,1,D1,26,1.000000,2,3,1,0.567164,0.745455,...,0,0,0,0,0,0,0,0,0,6
2147,2883,1,D2,26,0.487179,2,3,1,0.492537,0.690909,...,0,0,1,0,0,0,0,0,0,8
26716,35601,1,D3,26,0.846154,2,1,1,0.507463,0.781818,...,0,0,0,0,0,0,0,0,0,1
16324,21734,1,D2,10,0.230769,2,3,1,0.492537,0.636364,...,0,0,0,0,0,0,0,0,0,2


### Вычисление средних значений для каждой оценки
Проведем кластеризацию данных: разобьем их на группы по известной оценку скоринга (Response), вычислим центры этих групп как средние значения биометрических параметров.

In [3]:
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
responses = np.arange(1, data['Response'].max() + 1)
clusters = [{}]*(len(responses) + 1)
for r in responses:
    clusters[r] = {}
    for c in columns:
        clusters[r][c] = data[data['Response'] == r][c].median()
        
clusters

[{},
 {'Wt': 0.309623431,
  'Ht': 0.709090909,
  'Ins_Age': 0.52238806,
  'BMI': 0.483173533},
 {'Wt': 0.330543933,
  'Ht': 0.727272727,
  'Ins_Age': 0.47761194,
  'BMI': 0.510582282},
 {'Wt': 0.317991632,
  'Ht': 0.727272727,
  'Ins_Age': 0.358208955,
  'BMI': 0.510582282},
 {'Wt': 0.257322176,
  'Ht': 0.709090909,
  'Ins_Age': 0.328358209,
  'BMI': 0.4196545055},
 {'Wt': 0.351464435,
  'Ht': 0.709090909,
  'Ins_Age': 0.402985075,
  'BMI': 0.5918224864999999},
 {'Wt': 0.309623431,
  'Ht': 0.727272727,
  'Ins_Age': 0.432835821,
  'BMI': 0.489742491},
 {'Wt': 0.290794979,
  'Ht': 0.709090909,
  'Ins_Age': 0.447761194,
  'BMI': 0.470772683},
 {'Wt': 0.236401674,
  'Ht': 0.690909091,
  'Ins_Age': 0.313432836,
  'BMI': 0.394874563}]

### Выполним предсказание оценки скоринга на основе средних значений
Будем использовать евклидово расстояние:
\begin{equation}
D = \sqrt{ \sum {(a_i-С_i)^2}},\ где
\\a_i\ -\ значение\ параметров\ в\ проверочной\ выборке
\\C_i\ -\ значение\ центров\ кластеров\ по\ данным\ обучающей\ выборки
\end{equation}
Выберем принадлежность к кластеру, расстояние до которого минимально

In [4]:
def calc_model(x):
    D_min = 10000000
    target = 0
    for _, cluster in enumerate(clusters):
        if len(cluster) > 0:
            D = 0
            for c in columns:
                D += (cluster[c] - x[c])**2
            D = np.sqrt(D)
            if D < D_min:
                target = _
                D_min = D
    x['target'] = target
    x['random'] = int(np.random.uniform(1, 8.01, 1)[0])
    x['sample'] = data.sample(1)['Response'].values[0]
    x['all8'] = 8
    return x

In [5]:
data_test = data_test.apply(calc_model, axis='columns')
data_test.head(20)

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,target,random,sample,all8
8862,11808,1,D3,26,0.487179,2,3,1,0.223881,0.763636,...,0,0,0,0,0,2,3,7,1,8
20320,27073,1,A5,10,0.230769,2,3,1,0.358209,0.636364,...,0,0,0,0,0,5,5,6,7,8
16450,21892,1,E1,26,0.076923,2,3,1,0.462687,0.690909,...,0,0,0,0,0,8,7,7,7,8
53712,71504,1,D1,26,0.743590,2,3,3,0.537313,0.709091,...,0,0,0,0,0,7,1,2,4,8
16846,22446,1,D3,26,0.282051,2,3,1,0.373134,0.800000,...,0,0,0,0,0,6,3,7,2,8
19772,26353,1,D1,26,0.076923,2,3,1,0.641791,0.672727,...,0,0,0,0,0,2,1,4,8,8
53260,70910,1,D4,26,1.000000,2,1,1,0.373134,0.800000,...,0,0,0,0,0,6,4,5,8,8
21224,28264,1,A8,26,0.025641,2,1,1,0.582090,0.600000,...,0,0,1,0,0,1,5,3,7,8
23306,31075,1,E1,26,0.487179,2,3,1,0.134328,0.709091,...,0,0,0,0,0,8,8,4,3,8
25330,33735,1,A7,26,0.000000,2,3,1,0.462687,0.600000,...,0,0,0,0,0,5,8,5,5,8


### Оценка качества модели: F1
| скоринг \ исходные данные | 8 | 1 |
| --- | --- | --- |
| 8 | TP | FP |
| 1 | FN | TN |

\begin{equation}
Точность = \frac {TP} {TP + FP}
\end{equation}

\begin{equation}
Полнота = \frac {TP} {TP + FN}
\end{equation}

\begin{equation}
F1 = 2 * \frac {Точность * Полнота} {Точность + Полнота}
\end{equation}

In [6]:
print('Случайный выбор:', f1_score(data_test['random'], data_test['Response'], average='weighted'))
print('Выбор по частоте:', f1_score(data_test['sample'], data_test['Response'], average='weighted'))
print('Кластеризация:', f1_score(data_test['target'], data_test['Response'], average='weighted'))
print('Самый популярный:', f1_score(data_test['all8'], data_test['Response'], average='weighted'))

Случайный выбор: 0.09977209998916764
Выбор по частоте: 0.19350197365081576
Кластеризация: 0.2712309192201095
Самый популярный: 0.4884505249761375


### Матрица неточностей
| скоринг \ исходные данные | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 | FP6 | FP7 |
| 2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 | FP6 |
| 3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 |
| 4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 |
| 5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 |
| 6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 |
| 7 | FN6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 |
| 8 | FN7 | FN6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP |

In [8]:
print(confusion_matrix(data_test['target'], data_test['Response']))

[[ 538  467   30   46  291  672  584  779]
 [  64  100    8    8   65  201  119   70]
 [  91  115   36   32  159  361  310  200]
 [  68   65   23   39   35  219  157  442]
 [ 226  348   47    3  452  282   32    6]
 [  29   25    6    9    8   67   56   89]
 [  59   69   14   28   36  116  145  381]
 [ 146  115   38  114   79  321  266 1871]]


### Квадратичный коэффициент каппа Коэна
Является логичным продолжением оценке по матрице неточностей, но более точно указывает на соответствие вычисленных значений реальным, поскольку используем матрицу весов: большая ошибка получает больший вес.

Для расчета требуется вычислить матрицу весов (W), 8x8 выглядит так (каждый элемент - это квадрат разницы между номером строки и номером столба, разделенный на 64):

| матрица весов | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | 0 | 0.015625 | 0.0625 | 0.140625| 0.25 | 0.390625 | 0.5625 | 0.765625 |
| 2 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.140625 | 0.25 | 0.390625 | 0.5625 |
| 3 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 | 0.390625 | 0.5625 |
| 4 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 | 0.390625 |
| 5 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 |
| 6 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 |
| 7 | 0.5625 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 |
| 8 | 0.765625 | 0.5625 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 |

После вычисления матрицы неточностей (O) вычисляют матрицу гистограмм расчетных и идеальных значений (E) - сколько всего оценок 1, оценок 2, и т.д. В случае оценок от 1 до 8 гистограммы будут выглядеть следующим образом:

Расчет: \[3372, 661, 1244, 1040, 1380, 276, 900, 3004\]

Идеал: \[1193, 1302, 207, 261, 1120, 2257, 1633, 3904\]

Каждый элемент матрицы ij - это произведение i-расчетного значения на j-идеальное. Например, для ячейки 1-1 это будет 3372 * 1193 = 4022796. И т.д.

Матрицу неточностей и матрицу гистограмм нормируют (делят каждый элемент матрицы на сумму всех элементов) и вычисляют взвешенную сумму, используя матрицу весов (каждый элемент матрицы весов умножают на соответствующий элемент другой матрицы, все произведения суммируют): e = W * E, o = W * O.

Значение Kappa (каппа) вычисляется как 1 - o/e.

In [9]:
print(cohen_kappa_score(data_test['target'], data_test['Response'], weights='quadratic'))

0.2030830669811896


In [10]:
print(cohen_kappa_score(data_test['all8'], data_test['Response'], weights='quadratic'))

0.0
